In [1]:
import builtins
import copy
import time

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.common import StaleElementReferenceException

from itertools import combinations
from datetime import datetime

In [2]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

def get_timestamp():
    return str(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [3]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [4]:
wait = WebDriverWait(driver, 10)

In [5]:
driver.get("https://rustypot.com/coinflip")

In [6]:
def get_possible_bet_values(inventory_items_values, max_items=3):
    print(f"there are {len(inventory_items_values)} items in inventory with total value: {sum(inventory_items_values):.2f}$")
    possible_bet_values = {}
    for r in range(1, max_items + 1):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [7]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()
    
    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )
        
        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text.split("$")[1]))
        
        driver.get("https://rustypot.com/coinflip")

        print(f"inventory items gathered: {inventory_items_values}", )
        
        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        return get_inventory_items()

In [ ]:
inventory_items_values = get_inventory_items()
# inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34] ### TEST WITH HARDCODED ITEMS
start = time.time()
possible_bet_values = get_possible_bet_values(inventory_items_values)
end = time.time()

print(f"{end - start:.2f} seconds")
possible_bet_values

In [8]:
def get_bet_items(bet_values, coinflip_value):
    for bet_value in bet_values:
        if (coinflip_value * 0.9 < bet_value <= coinflip_value * max_bet_percentage and 
                # coinflip_value not in bets and 
                min_bet_value < coinflip_value < max_bet_value):
            return bet_values[bet_value]
    
    return None


def join_coinflip(join_button):

    try:
        join_button.click()
        
        spinner_locator = (By.CLASS_NAME, "spinner")
        WebDriverWait(driver, 4).until(
            EC.invisibility_of_element_located(spinner_locator)
        )
    except TimeoutException:
        print(f"Inventory loading timed out during selecting bet items")
        driver.get("https://rustypot.com/coinflip")
        join_coinflip(join_button)
    
    
def select_bet_items(bet_items_values):
    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
    inventory_items = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located(inventory_items_locator)
    )
    
    for inventory_item in inventory_items:
        item_value = float(inventory_item.text.split("$")[1])
        if item_value in bet_items_values:
            inventory_item.click()
            bet_items_values.remove(item_value)
    

def press_deposit_button():
    deposit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
    )
    deposit_button.click()
    
    
def press_accept_deposit_button_site():
    accept_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
    )
    accept_button.click()

def press_accept_winnings_button_site():
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH,
                                "//button[@onclick='closeAcceptWinnings()']"))
    )
    accept_button.click()

def press_no_doubledown_button():
    cancel_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@id='DDCancel' and contains(@class, 'DDOption')]"))
    )
    cancel_button.click()


def accept_steam_trade(deposit, acceptWinning):
    
    # 1.open the new window
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                
    original_window = driver.current_window_handle
    new_window = [w for w in driver.window_handles if w != original_window][0]
    driver.switch_to.window(new_window)
    
    
    # 2.close if profile reported    
    try:
        warning_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//*[contains(., 'have reported Blown for attempting trade scams')]"
            ))
        )
        ok_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                ))
        )
        ok_button.click()
    except TimeoutException:
        pass
    
    # 3. Accept the trade
    confirm_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
    )
    confirm_button.click()
    time.sleep(0.25)
    
    
    
    if deposit:
        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)
    elif acceptWinning:
        
        
        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='btn_green_steamui btn_medium']"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)
        

        try:
            trust_bot_element = WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.XPATH, "//div[span[contains(text(), 'Yes, I trust')]]"))
            )
            trust_bot_element.click()
            time.sleep(0.25)
        except TimeoutException:
            pass
    
    accept_trade_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
    )
    accept_trade_btn.click()
    time.sleep(1)

    driver.close()
    driver.switch_to.window(original_window)
    
    
def open_cf_window_and_wait_to_flip():
    view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
    view_button.click()
    
    # Wait until the <p> inside #fliper-coin disappears
    try:
        p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
        WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
    except:
        # If <p> is already gone, just wait for the new state
        pass
        
    
    WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
    )


def get_winner():
    # Wait for the parent div to appear
    creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))
    
    # Wait for the child img.pick within that div
    color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))
    
    # Get the src
    src = color_img.get_attribute("src")
    
     # Extract the color from the filename
    if "Red" in src:
        creator_color = "Red"
    elif "Black" in src:
        creator_color = "Black"
    else:  # creator_color = "Unknown"
        time.sleep(0.5)
        print(f"creator_color is {src}. Retry getting creator")
        return get_winner()
    
    # 1. Get the 'flipper' element
    flipper = driver.find_element(By.CLASS_NAME, "flipper")
    
    # 2. Get the 'transform' style attribute
    transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"
    
    # 3. Extract the degree
    import re
    match = re.search(r'rotateY\((\d+)deg\)', transform_style)
    if match:
        degrees = int(match.group(1))
        # Normalize the rotation within 360°
        final_angle = degrees % 360
    
        # Every 180° flips the face, so even number of 180s = same side
        flips = (degrees // 180) % 2
    
        flip_face = "Red" if flips == 1 else "Black"
        print("Coin landed on:", flip_face)
    else:
        time.sleep(0.5)
        print("Couldn't determine coin face. Retry getting flipping face")
        return get_winner()
        
    return creator_color, flip_face

def get_coinflip_items_values(coinflip_items):
    img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
    coinflip_items_values = []
    for img in img_elements:
        title = img.get_attribute("data-original-title")
        if "|" in title:
            price_str = title.split("$")[-1]
            price = float(price_str)
            coinflip_items_values.append(price)
    
    return coinflip_items_values

def log_bet(bet_value, won_value, win, profit, inventory_value):
    f = open("logs.csv", "a")
    f.write(f"{get_timestamp()},"
            f"{bet_value:.2f},"
            f"{won_value:.2f},"
            f"{win},"
            f"{profit:.2f},"
            f"{inventory_value:.2f}\n")
    print(f"LOG APPEND: {get_timestamp()},"
            f"{bet_value:.2f},"
            f"{won_value:.2f},"
            f"{win},"
            f"{profit:.2f},"
            f"{inventory_value:.2f}")
    f.close()

In [14]:
max_bet_percentage = 0.94
tax_procentage = 0.05
min_bet_value = 1.0
max_bet_value = 4 * (1 + (1 - max_bet_percentage))
# max_bet_value = 4

In [20]:
inventory_items_values = get_inventory_items()
possible_bet_values = get_possible_bet_values(inventory_items_values)

while True:
    try:
        active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
        coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
        
        for coinflip_div in coinflip_row_divs:
            try:
                coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
                coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
            except StaleElementReferenceException:
                print(f"coinflip divs are stale. Retry getting coinflip rows")
                break
            
            # cf value + bet value ( max 95% of cf value)
            tax_value = (coinflip_value + coinflip_value * max_bet_percentage) * tax_procentage  # the taxed amount is the total amount of the coinflip
    
            if "items" in coinflip_items.text:
                # print("more than 5 items...")
                continue
    
            coinflip_items_values = get_coinflip_items_values(coinflip_items)
    
            if any(tax_value > item_value for item_value in coinflip_items_values):
                # print(f"not profitable")
                continue
            
            
            # to join a cf, i need to deposit minimum value
            # 1. get the bet items
            bet_items_values = get_bet_items(possible_bet_values, coinflip_value)
            
            if bet_items_values is None:
                # no available bet for this cf based on our items
                continue
            
            ### Press join on available coinflip. if it is ongoing (only View button visible, skip)
            join_button = coinflip_div.find_element(By.TAG_NAME, "button")
            if "Join" not in join_button.text:
                continue
                
            print(f"JOINING bet_value = {sum(bet_items_values):.2f} -> bet_items_value{bet_items_values=}")
    
            join_coinflip(join_button)
            
            try:
                select_bet_items(copy.deepcopy(bet_items_values))
            except TimeoutException:
                print(f"Timed out when selecting bet items")
                driver.get("https://rustypot.com/coinflip")
                time.sleep(0.5)
                break
                
            press_deposit_button()
            
            press_accept_deposit_button_site()

            # try:
            #     # check if game already in progress
            #     element = driver.find_element(By.XPATH, '//div[@id="AcceptDepositBody"]/h4[text()="This game is already in progress!"]')
            # 
            #     WebDriverWait(driver, 3).until(
            #         EC.presence_of_element_located(element)
            #     )
            #     
            #     # if present, break and continue
            #     driver.get("https://rustypot.com/coinflip")
            #     time.sleep(0.5)
            #     break
            # except TimeoutException:
            #     pass
            
            
            accept_steam_trade(deposit=True, acceptWinning=False)
    
     
            print(f"JOINED {coinflip_value=} -> {coinflip_items_values=}")
                
            # bets.append(coinflip_value)
            
            open_cf_window_and_wait_to_flip()
    
            creator_color, flip_face = get_winner()
    
            # extract flip color
            
            if creator_color == flip_face:
                print(f"flip was lost!")
                time.sleep(4) # wait to see the flip
    
                driver.get("https://rustypot.com/coinflip")
                time.sleep(1)
                for item in bet_items_values:
                    inventory_items_values.remove(item)
                    
                log_bet(sum(bet_items_values), coinflip_value, 0, -sum(bet_items_values), sum(inventory_items_values))
                
                possible_bet_values = get_possible_bet_values(inventory_items_values)
                
            else:
                print(f"flip was won!")
                time.sleep(4)
                
                press_no_doubledown_button()
    
                press_accept_winnings_button_site()
    
                accept_steam_trade(deposit=False, acceptWinning=True)
                
                for item in coinflip_items_values:
                    inventory_items_values.append(item)
                    
                log_bet(sum(bet_items_values), coinflip_value, 1, coinflip_value, sum(inventory_items_values))
    
                possible_bet_values = get_possible_bet_values(inventory_items_values)
        
            print(f"Back to searching")
    except Exception:
        print(f"stale element error -> maybe captch / cf already started / something || RESET")
        driver.get("https://rustypot.com/coinflip")

    time.sleep(1)

[2025-03-25 16:32:32]  inventory items gathered: [7.02, 5.3, 5.26, 4.96, 4.76, 4.38, 4.28, 4.28, 4.09, 4.08, 4.03, 3.92, 3.9, 3.73, 3.72, 3.71, 3.71, 3.64, 3.58, 3.47, 3.34, 3.34, 3.26, 3.15, 3.09, 3.0, 2.99, 2.99, 2.98, 2.82, 2.63, 2.63, 2.55, 2.47, 2.35, 2.32, 2.25, 2.25, 2.21, 2.1, 2.1, 2.09, 2.0, 2.0, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.99, 1.77, 1.73, 1.64, 1.63, 1.63, 1.61, 1.55, 1.54, 1.05, 1.04, 0.99, 0.94, 0.69]
[2025-03-25 16:32:32]  there are 64 items in inventory with total value: 180.47$
[2025-03-25 16:32:33]  JOINING bet_value = 0.99 -> bet_items_valuebet_items_values=[0.99]
[2025-03-25 16:32:38]  JOINED coinflip_value=1.07 -> coinflip_items_values=[1.07]
[2025-03-25 16:32:56]  Coin landed on: Black
[2025-03-25 16:32:56]  flip was won!
[2025-03-25 16:33:08]  LOG APPEND: 2025-03-25 16:33:08,0.99,1.07,1,1.07,181.54
[2025-03-25 16:33:08]  there are 65 items in inventory with total value: 181.54$
[2025-03-25 16:33:08]  Back to searching
[2025-03-25 16:33:10]  JOINING bet_va

KeyboardInterrupt: 

In [ ]:
press_no_doubledown_button()
press_accept_winnings_button_site()
accept_steam_trade(deposit=False, acceptWinning=True)

In [70]:
driver.switch_to.window(driver.window_handles[0])

In [34]:
driver.window_handles

['72E98C09411D25942DFB48696CE87804']

In [20]:
confirm_button = WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
)
confirm_button.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x011B0E93+39155]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x0104E3B6+772070]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00E048EE+5182]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E3998A+105610]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E39C7A+106362]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E69B92+302738]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E51EC4+205252]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E33091+78737]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E68530+297008]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E51B86+204422]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E32947+76871]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E31CF5+73717]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00E32764+76388]
	sqlite3_dbdata_init [0x012B65BC+589708]
	Microsoft::Applications::Events:

In [100]:
log_bet(2.92, 
        3.24,
        0,
        2.92, 
        248.74 - 2.92)

[2025-03-24 21:53:33]  LOG APPEND: 2025-03-24 21:53:33,2.92,3.24,0,2.92,245.82


In [62]:
trust_bot_element = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//div[span[contains(text(), 'Yes, I trust')]]"))
)
trust_bot_element.click()